# 8장. 생성 모델을위한 딥러닝

## 8.1 LSTM으로 텍스트 생성하기
### 시퀀스 데이터를 어떻게 생성할까?
딥러닝에서 시퀀스 데이터를 생성하는 일반적인 방법은 이전 토큰을 입력으로 사용해서 시퀀스의 다음 1개 또는 몇 개의 토큰을 예측하는 것  
이때, 이전 토큰들이 주어졌을 때 다음 토큰의 확률을 모델링할 수 있는 네트워크를 **언어 모델** 이라고 부름.  
언어 모델은 언어의 통계적 구조인 잠개 공간을 탐색 함  

언어 모델을 훈련하고 나면 이 모델에서 샘플링을 할 수 있음  
**조건 데이터**(초기 텍스트 문자열)을 주입하고 새로운 글자와 단어 생성  
생성된 출력은 다시 입력데이터로 추가  
이 과정을 여러번 반복  

### 샘플링 전략의 중요성
샘플링 종류 
- 탐욕적 샘플링 : 가장 높은 확률을 가진 글자를 선택으로 논리적인 언어를 만드는데 적합하지 않음
- 확률적 샘플링 : 글자의 확률 분포에서 샘플링하는 과정에 무작위성을 주입하는 방법으로 소프트맥스 온도 파라미터를 사용해 확률 분포 엔트로피를 조절  



In [2]:
# 다른 온도 값을 사용하여 확률 분포의 가중치 바꾸기
import numpy as np

def reweight_distribution(original_distribution, temperature=0.5):
    distribution = np.log(original_distribution) / temperature
    distribution = np.exp(distribution)
    return distribution / np.sum(distribution)

### 글자 수준의 LSTM 텍스트 생성 모델 구현
#### 데이터 전처리

In [1]:
# 원본 텍스트 파일을 내려받아 파싱하기
import keras
import numpy as np

path = keras.utils.get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
text = open(path).read().lower()
print('length  of corpus : ', len(text))

606208/600901 [==============================] - 0s 0us/step
length  of corpus :  600893


In [3]:
# 글자 시퀀스 벡터화하기
maxlen = 60
step = 3

sentences = []
next_chars = []

for i in range(0, len(text) - maxlen, step):
  sentences.append(text[i:i+maxlen])
  next_chars.append(text[i+maxlen])

print('시퀀스 개수: ', len(sentences))

chars = sorted(list(set(text)))
print('고유한 글자: ', len(chars))
char_indices = dict((char, chars.index(char)) for char in chars)

print('Vectorize...')

x = np.zeros((len(sentences),maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
  for t, char in enumerate(sentence):
    x[i, t, char_indices[char]] = 1
  y[i, char_indices[next_chars[i]]] = 1

시퀀스 개수:  200278
고유한 글자:  57
Vectorize...


#### 네트워크 구성
LSTM 층과 Dense 분류기를 이용한 네트워크를 만듦  


In [4]:
# 다음 글자를 예측하기 위한 단일 LSTM 모델
from keras import layers

model = keras.models.Sequential()
model.add(layers.LSTM(128, input_shape=(maxlen, len(chars))))
model.add(layers.Dense(len(chars), activation='softmax'))

In [5]:
# 모델 컴파일 설정하기
optimizer = keras.optimizers.RMSprop(lr=0.01)
model.compile(optimizer = optimizer,
              loss = 'categorical_crossentropy')

#### 언어 모델 훈련과 샘플링
훈련된 모델과 시드로 쓰일 간단한 텍스트가 주어지면 다음과 같이 반복해 새로운 텍스트를 생성할 수 있음  

1. 지금까지 생성된 텍스트를 주입하여 모델에서 다음 글자에 대한 확률 분포를 뽑음
2. 특정 온도로 이 확률 분포의 가중치를 조정
3. 가중치가 조정된 분포에서 무작위로 새로운 글자 샘플링
4. 새로운 글자를 생성된 텍스트의 끝에 추가

In [6]:
# 모델의 예측이 주어졌을 때 새로운 글자를 샘플링하는 함수
def sample(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [ ]:
# 텍스트 생성 루프
import random
import sys

random.seed(42)
start_index = random.randint(0, len(text) - maxlen - 1)

for epoch in range(1, 60):
    print('에포크', epoch)
    model.fit(x, y, batch_size=128, epochs=1)

    seed_text = text[start_index: start_index + maxlen]
    print('--- 시드 텍스트: "' + seed_text + '"')

    for temperature in [0.2, 0.5, 1.0, 1.2]:
        print('------ 온도:', temperature)
        generated_text = seed_text
        sys.stdout.write(generated_text)

        for i in range(400):
            sampled = np.zeros((1, maxlen, len(chars)))
            for t, char in enumerate(generated_text):
                sampled[0, t, char_indices[char]] = 1.

            preds = model.predict(sampled, verbose=0)[0]
            next_index = sample(preds, temperature)
            next_char = chars[next_index]

            generated_text += next_char
            generated_text = generated_text[1:]

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

낮은 온도에선 반복적이고 예상되는 텍스트를 만들지만, 국부적인 구조는 실제와 거의 유사  

높은 온도에선 국부적인 구조가 무너지기 시작하며 대부분의 단어가 어느 정도 무작위한 문자열로 보임  

## 8.2 딥드림
**합성곱 신경망이 학습한 표현을 사용하여 예술적으로 이미지를 조작하는 기법**  

딥드림 구현 아이디어
- 특정 필터가 아니라 전체 층의 활성화를 최대화  
- 빈 이미지나 노이즈가 조금 있는 입력이 아니라 이미 가지고 있는 이미지를 사용
- 입력 이미지는 시각 품질을 높이기 위해 여러 다른 스케일로 처리  

### 케라스 딥드림 구현


In [1]:
!pip uninstall keras -y
!pip install keras==2.2.4

Uninstalling Keras-2.4.3:
  Successfully uninstalled Keras-2.4.3
     |████████████████████████████████| 317kB 7.7MB/s 
     |████████████████████████████████| 51kB 7.8MB/s 


In [ ]:
import tensorflow as tf

tf.compat.v1.disable_v2_behavior()

In [3]:
# 사전 훈련된 인셉션 V3 모델 로드하기
from tensorflow.keras.applications import inception_v3
from tensorflow.keras import backend as K

K.set_learning_phase(0)

model = inception_v3.InceptionV3(weights='imagenet',
                                 include_top=False)

/usr/local/lib/python3.6/dist-packages/tensorflow/python/keras/backend.py:434: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn('`tf.keras.backend.set_learning_phase` is deprecated and '


87916544/87910968 [==============================] - 1s 0us/step


In [2]:
model.summary()

Model: "inception_v3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
_______________________________________________________________________________________

모델 층에 이름과 계수를 매핑

In [4]:
# 딥드림 설정하기
layer_contributions = {
    'mixed2' : 0.2,
    'mixed3' : 3.,
    'mixed4' : 2.,
    'mixed5' : 1.5,
}

In [5]:
# 최대화할 손실 정의하기
layer_dict = dict([layer.name, layer] for layer in model.layers)

loss = K.variable(0.)
for layer_name in layer_contributions:
    coeff = layer_contributions[layer_name]
    activation = layer_dict[layer_name].output

    scaling = K.prod(K.cast(K.shape(activation), 'float32'))
    loss = loss + coeff * K.sum(K.square(activation[:, 2:-2, 2:-2, :]))

In [6]:
# 경사 상승법 과정
dream = model.input
grads = K.gradients(loss, dream)[0]
grads /= K.maximum(K.mean(K.abs(grads)), 1e-7)

outputs = [loss, grads]
fetch_loss_and_grads = K.function([dream], outputs)

def eval_loss_and_grads(x):
    outs = fetch_loss_and_grads([x])
    loss_value = outs[0]
    grad_values = outs[1]
    return loss_value, grad_values

def gradient_ascent(x, iterations, step, max_loss=None):
    for i in range(iterations):
        loss_value, grad_values = eval_loss_and_grads(x)
        if max_loss is not None and loss_value > max_loss:
            break
        print('...', i, '번째 손실 :', loss_value)
        x += step * grad_values
    return x

경사 상승법을 실행 한 후 크기를 40% 증가

In [7]:
from google.colab import files
files.upload()

Saving original_photo_deep_dream.jpg to original_photo_deep_dream.jpg


{'original_photo_deep_dream.jpg': b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00H\x00H\x00\x00\xff\xe1\x00\xc8Exif\x00\x00MM\x00*\x00\x00\x00\x08\x00\x07\x01\x12\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\x01\x1a\x00\x05\x00\x00\x00\x01\x00\x00\x00b\x01\x1b\x00\x05\x00\x00\x00\x01\x00\x00\x00j\x01(\x00\x03\x00\x00\x00\x01\x00\x02\x00\x00\x011\x00\x02\x00\x00\x00\x0f\x00\x00\x00r\x012\x00\x02\x00\x00\x00\x14\x00\x00\x00\x82\x87i\x00\x04\x00\x00\x00\x01\x00\x00\x00\x96\x00\x00\x00\x00\x00\x00\x00H\x00\x00\x00\x01\x00\x00\x00H\x00\x00\x00\x01Pixelmator 3.3\x00\x002018:08:20 15:08:43\x00\x00\x03\xa0\x01\x00\x03\x00\x00\x00\x01\x00\x01\x00\x00\xa0\x02\x00\x04\x00\x00\x00\x01\x00\x00\x01^\xa0\x03\x00\x04\x00\x00\x00\x01\x00\x00\x01^\x00\x00\x00\x00\xff\xe1\t\x90http://ns.adobe.com/xap/1.0/\x00<?xpacket begin="\xef\xbb\xbf" id="W5M0MpCehiHzreSzNTczkc9d"?> <x:xmpmeta xmlns:x="adobe:ns:meta/" x:xmptk="XMP Core 5.4.0"> <rdf:RDF xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"> <rdf:Descr

In [9]:
# 유틸리티 함수
import scipy
from keras.preprocessing import image

def resize_img(img, size):
    img = np.copy(img)
    factors = (1,
               float(size[0]) / img.shape[1],
               float(size[1]) / img.shape[2],
               1)
    return scipy.ndimage.zoom(img, factors, order=1)


def save_img(img, fname):
    pil_img = deprocess_image(np.copy(img))
    image.save_img(fname, pil_img)


def preprocess_image(image_path):
    img = image.load_img(image_path)
    img = image.img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = inception_v3.preprocess_input(img)
    return img


def deprocess_image(x):
    if K.image_data_format() == 'channels_first':
        x = x.reshape((3, x.shape[2], x.shape[3]))
        x = x.transpose((1, 2, 0))
    else:
        x = x.reshape((x.shape[1], x.shape[2], 3))
    x /= 2.
    x += 0.5
    x *= 255.
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [11]:
# 연속적인 스케일에 걸쳐 경사 상승법 실행하기
import numpy as np

step = 0.01
num_octave = 3
octave_scale = 1.4
iterations = 20

max_loss = 10.
base_image_path = "/content/original_photo_deep_dream.jpg"

img = preprocess_image(base_image_path)

original_shape = img.shape[1:3]
successive_shapes = [original_shape]
for i in range(1, num_octave):
    shape = tuple([int(dim / (octave_scale ** i)) for dim in original_shape])
    successive_shapes.append(shape)

successive_shapes = successive_shapes[::-1]

original_img = np.copy(img)
shrunk_original_img = resize_img(img, successive_shapes[0])

for shape in successive_shapes:
    print('처리할 이미지 크기', shape)
    img = resize_img(img, shape)
    img = gradient_ascent(img,
                          iterations=iterations,
                          step=step,
                          max_loss=max_loss)
    upscaled_shrunk_original_img = resize_img(shrunk_original_img, shape)
    same_size_original = resize_img(original_img, shape)
    lost_detail = same_size_original - upscaled_shrunk_original_img

    img += lost_detail
    shrunk_original_img = resize_img(original_img, shape)
    save_img(img, fname='dream_at_scale_' + str(shape) + '.png')

save_img(img, fname='/content/final_dream.png')

처리할 이미지 크기 (178, 178)
처리할 이미지 크기 (250, 250)
처리할 이미지 크기 (350, 350)


## 8.3 뉴럴 스타일 트랜스퍼
**타깃 이미지의 콘텐츠를 보존하면서 참조 이미지의 스타일을 타깃 이미지에 적용하는 기법**  